# Chapter 4 - Exercise 6: Scaffold Architecture and Porosity Design

## Learning Objectives
- Understand the relationship between porosity and scaffold properties
- Calculate permeability and diffusion characteristics
- Apply Gibson-Ashby model for mechanical properties
- Optimize pore size for cell migration and vascularization
- Design hierarchical porous structures
- Balance mechanical strength with biological function

## Background

**Scaffold architecture** is critical for tissue engineering success. The porous structure must:
- **Support cell infiltration** (pore size >100 µm)
- **Enable nutrient/waste transport** (interconnected pores)
- **Allow vascularization** (pore size 200-500 µm)
- **Provide mechanical support** (conflicts with high porosity)
- **Guide tissue formation** (pore shape and orientation)

### Key Equations (from Chapter 4.3):

**Porosity**:
$$\varepsilon = \frac{V_{void}}{V_{total}} = 1 - \frac{\rho_{scaffold}}{\rho_{solid}}$$

**Gibson-Ashby Model** (mechanical properties):
$$\frac{E_{scaffold}}{E_{solid}} = C \left(\frac{\rho_{scaffold}}{\rho_{solid}}\right)^n = C(1-\varepsilon)^n$$

For open-cell foams: n ≈ 2, C ≈ 1

**Carman-Kozeny Equation** (permeability):
$$k = \frac{d_p^2 \varepsilon^3}{180(1-\varepsilon)^2}$$

where:
- k = permeability (m²)
- d_p = pore diameter (m)
- ε = porosity (dimensionless)

**Diffusion Length**:
$$L = \sqrt{4Dt}$$

Critical for cell survival: L ≤ 100-200 µm (oxygen diffusion limit)

### Design Guidelines:
- **Bone**: 50-70% porosity, 200-500 µm pores
- **Cartilage**: 70-90% porosity, 100-200 µm pores
- **Soft tissue**: 80-95% porosity, 50-150 µm pores
- **Vascularization**: requires >200 µm pores

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import fsolve, minimize
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Circle, Rectangle
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready to design scaffold architectures!")

## Part 1: Porosity and Mechanical Property Functions

In [ ]:
def calculate_porosity(scaffold_density, solid_density):
    """
    Calculate porosity from density ratio.
    
    Parameters:
    -----------
    scaffold_density : float
        Apparent density of porous scaffold (kg/m³)
    solid_density : float
        Density of solid material (kg/m³)
    
    Returns:
    --------
    porosity : float
        Porosity (0-1)
    """
    porosity = 1 - (scaffold_density / solid_density)
    return porosity

def gibson_ashby_modulus(E_solid, porosity, C=1, n=2):
    """
    Calculate scaffold elastic modulus using Gibson-Ashby model.
    
    Parameters:
    -----------
    E_solid : float
        Elastic modulus of solid material (Pa or MPa)
    porosity : float
        Porosity (0-1)
    C : float
        Geometric constant (~1 for open-cell foams)
    n : float
        Exponent (~2 for open-cell, ~1.5 for closed-cell)
    
    Returns:
    --------
    E_scaffold : float
        Scaffold elastic modulus (same units as E_solid)
    """
    relative_density = 1 - porosity
    E_scaffold = E_solid * C * (relative_density ** n)
    return E_scaffold

def gibson_ashby_strength(sigma_solid, porosity, C=0.3, n=1.5):
    """
    Calculate scaffold strength using Gibson-Ashby model.
    
    Parameters:
    -----------
    sigma_solid : float
        Strength of solid material (Pa or MPa)
    porosity : float
        Porosity (0-1)
    C : float
        Geometric constant (~0.3 for open-cell foams)
    n : float
        Exponent (~1.5 for open-cell)
    
    Returns:
    --------
    sigma_scaffold : float
        Scaffold strength (same units as sigma_solid)
    """
    relative_density = 1 - porosity
    sigma_scaffold = sigma_solid * C * (relative_density ** n)
    return sigma_scaffold

def carman_kozeny_permeability(pore_diameter_um, porosity):
    """
    Calculate permeability using Carman-Kozeny equation.
    
    Parameters:
    -----------
    pore_diameter_um : float
        Average pore diameter (µm)
    porosity : float
        Porosity (0-1)
    
    Returns:
    --------
    permeability : float
        Hydraulic permeability (m²)
    """
    dp_m = pore_diameter_um * 1e-6  # Convert to meters
    
    k = (dp_m**2 * porosity**3) / (180 * (1 - porosity)**2)
    
    return k

def calculate_flow_rate(permeability, pressure_drop, length, viscosity, area):
    """
    Calculate flow rate through porous scaffold (Darcy's law).
    
    Parameters:
    -----------
    permeability : float
        Hydraulic permeability (m²)
    pressure_drop : float
        Pressure difference (Pa)
    length : float
        Flow path length (m)
    viscosity : float
        Fluid dynamic viscosity (Pa·s)
    area : float
        Cross-sectional area (m²)
    
    Returns:
    --------
    flow_rate : float
        Volumetric flow rate (m³/s)
    """
    # Darcy's law: Q = (k·A·ΔP) / (η·L)
    Q = (permeability * area * pressure_drop) / (viscosity * length)
    return Q

def oxygen_diffusion_distance(time_hours, D=2e-9):
    """
    Calculate oxygen diffusion distance.
    
    Parameters:
    -----------
    time_hours : float
        Time (hours)
    D : float
        Oxygen diffusion coefficient in tissue (m²/s)
        Default: 2e-9 m²/s
    
    Returns:
    --------
    distance_um : float
        Diffusion distance (µm)
    """
    t_s = time_hours * 3600  # Convert to seconds
    L = np.sqrt(4 * D * t_s)  # meters
    L_um = L * 1e6  # Convert to µm
    return L_um

def assess_pore_size(pore_size_um, application='general'):
    """
    Assess if pore size is appropriate for application.
    
    Parameters:
    -----------
    pore_size_um : float
        Pore diameter (µm)
    application : str
        'bone', 'cartilage', 'soft_tissue', 'vascular', or 'general'
    
    Returns:
    --------
    dict : Assessment with recommendations
    """
    assessment = {
        'suitable': False,
        'issues': [],
        'recommendations': []
    }
    
    # Define optimal ranges
    ranges = {
        'bone': (200, 500),
        'cartilage': (100, 200),
        'soft_tissue': (50, 150),
        'vascular': (200, 500),
        'general': (100, 300)
    }
    
    min_size, max_size = ranges.get(application, ranges['general'])
    
    if min_size <= pore_size_um <= max_size:
        assessment['suitable'] = True
    elif pore_size_um < min_size:
        assessment['issues'].append(f'Pore too small for {application}')
        assessment['recommendations'].append(f'Increase to {min_size}-{max_size} µm')
        
        if pore_size_um < 100:
            assessment['issues'].append('May restrict cell migration')
        if application == 'vascular' and pore_size_um < 200:
            assessment['issues'].append('Insufficient for blood vessel ingrowth')
    else:
        assessment['issues'].append(f'Pore too large for {application}')
        assessment['recommendations'].append(f'Reduce to {min_size}-{max_size} µm')
        assessment['issues'].append('May compromise mechanical strength')
    
    return assessment

print("✓ Scaffold architecture calculation functions defined!")
print("\nKey functions:")
print("  • calculate_porosity()")
print("  • gibson_ashby_modulus()")
print("  • gibson_ashby_strength()")
print("  • carman_kozeny_permeability()")
print("  • oxygen_diffusion_distance()")
print("  • assess_pore_size()")

## Part 2: Gibson-Ashby Model Visualization

Understand how porosity affects mechanical properties:

In [ ]:
# Create porosity range
porosity_range = np.linspace(0, 0.95, 100)

# Material properties (example: PCL)
E_solid = 400  # MPa
sigma_solid = 16  # MPa

# Calculate scaffold properties
E_scaffold = gibson_ashby_modulus(E_solid, porosity_range, C=1, n=2)
sigma_scaffold = gibson_ashby_strength(sigma_solid, porosity_range, C=0.3, n=1.5)

# Native tissue properties for comparison
tissues = {
    'Bone (cortical)': {'E': 15000, 'sigma': 150, 'color': '#e74c3c'},
    'Bone (trabecular)': {'E': 100, 'sigma': 5, 'color': '#f39c12'},
    'Cartilage': {'E': 10, 'sigma': 5, 'color': '#3498db'},
    'Soft tissue': {'E': 0.5, 'sigma': 0.5, 'color': '#2ecc71'}
}

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Modulus vs Porosity
ax1.semilogy(porosity_range * 100, E_scaffold, linewidth=3, color='#9b59b6',
            label='PCL Scaffold (Gibson-Ashby)')

# Add tissue ranges
for tissue_name, props in tissues.items():
    ax1.axhline(props['E'], color=props['color'], linestyle='--', linewidth=2,
               alpha=0.7, label=tissue_name)

ax1.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Elastic Modulus (MPa)', fontsize=12, fontweight='bold')
ax1.set_title('Gibson-Ashby: Modulus vs Porosity\nE/E₀ = (1-ε)²',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax1.grid(True, alpha=0.3, which='both')
ax1.set_ylim(0.1, 1000)

# Plot 2: Strength vs Porosity
ax2.semilogy(porosity_range * 100, sigma_scaffold, linewidth=3, color='#e74c3c',
            label='PCL Scaffold')

# Add tissue strength ranges
for tissue_name, props in tissues.items():
    ax2.axhline(props['sigma'], color=props['color'], linestyle='--',
               linewidth=2, alpha=0.7, label=tissue_name)

ax2.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Compressive Strength (MPa)', fontsize=12, fontweight='bold')
ax2.set_title('Gibson-Ashby: Strength vs Porosity\nσ/σ₀ = 0.3(1-ε)^1.5',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax2.grid(True, alpha=0.3, which='both')
ax2.set_ylim(0.01, 20)

# Plot 3: Linear scale to see detail
# Zoom in on typical scaffold range
mask = porosity_range <= 0.8
ax3.plot(porosity_range[mask] * 100, E_scaffold[mask], linewidth=3,
        color='#9b59b6', label='Elastic Modulus')
ax3_twin = ax3.twinx()
ax3_twin.plot(porosity_range[mask] * 100, sigma_scaffold[mask], linewidth=3,
             color='#e74c3c', linestyle='--', label='Strength')

# Highlight typical design range
ax3.axvspan(50, 70, alpha=0.2, color='green', label='Typical design range')

ax3.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Elastic Modulus (MPa)', fontsize=12, fontweight='bold', color='#9b59b6')
ax3_twin.set_ylabel('Compressive Strength (MPa)', fontsize=12, fontweight='bold',
                   color='#e74c3c')
ax3.set_title('Mechanical Properties (Linear Scale)\nTypical Scaffold Range',
             fontsize=13, fontweight='bold')
ax3.tick_params(axis='y', labelcolor='#9b59b6')
ax3_twin.tick_params(axis='y', labelcolor='#e74c3c')
ax3.grid(True, alpha=0.3)

# Plot 4: Ashby plot (strength vs modulus)
ax4.loglog(E_scaffold, sigma_scaffold, linewidth=3, color='#9b59b6',
          label='PCL Scaffold\n(varying porosity)', marker='o', markevery=10,
          markersize=8)

# Plot native tissues
for tissue_name, props in tissues.items():
    ax4.plot(props['E'], props['sigma'], 'o', markersize=15,
            color=props['color'], markeredgecolor='black', markeredgewidth=2,
            label=tissue_name)

# Add porosity labels on scaffold curve
porosity_labels = [0.3, 0.5, 0.7, 0.9]
for p in porosity_labels:
    E = gibson_ashby_modulus(E_solid, p)
    sigma = gibson_ashby_strength(sigma_solid, p)
    ax4.annotate(f'ε={p:.0%}', xy=(E, sigma), xytext=(10, 10),
                textcoords='offset points', fontsize=9, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

ax4.set_xlabel('Elastic Modulus (MPa)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Compressive Strength (MPa)', fontsize=12, fontweight='bold')
ax4.set_title('Ashby Plot: Strength vs Modulus\n(Material Selection Chart)',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True, fontsize=9)
ax4.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print("\n💡 Gibson-Ashby Insights:")
print("   • Modulus decreases as (1-ε)² → quadratic relationship")
print("   • Strength decreases as (1-ε)^1.5 → slightly less sensitive")
print("   • At 50% porosity: E = 25% of solid, σ = 21% of solid")
print("   • At 70% porosity: E = 9% of solid, σ = 8% of solid")
print("   • Trade-off: High porosity (good for biology) vs strength")
print("   • Bone scaffolds need 50-70% porosity to match trabecular bone")

## Part 3: Permeability and Mass Transport Analysis

In [ ]:
# Create parameter ranges
pore_sizes = np.linspace(50, 500, 100)  # µm
porosities_perm = [0.5, 0.6, 0.7, 0.8, 0.9]
colors_perm = ['#8e44ad', '#3498db', '#2ecc71', '#f39c12', '#e74c3c']

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Permeability vs pore size for different porosities
for porosity, color in zip(porosities_perm, colors_perm):
    permeabilities = [carman_kozeny_permeability(d, porosity) for d in pore_sizes]
    permeabilities_m2 = np.array(permeabilities) * 1e12  # Convert to 10^-12 m²
    
    ax1.semilogy(pore_sizes, permeabilities_m2, linewidth=3, color=color,
                label=f'ε = {porosity:.0%}')

ax1.set_xlabel('Pore Diameter (µm)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Permeability (×10⁻¹² m²)', fontsize=12, fontweight='bold')
ax1.set_title('Carman-Kozeny: Permeability vs Pore Size\nk ∝ d²·ε³/(1-ε)²',
             fontsize=13, fontweight='bold')
ax1.legend(loc='best', frameon=True, fancybox=True)
ax1.grid(True, alpha=0.3, which='both')

# Plot 2: Permeability vs porosity for different pore sizes
porosity_fine = np.linspace(0.3, 0.95, 100)
pore_sizes_select = [100, 200, 300, 400]  # µm
colors_pore = ['#8e44ad', '#3498db', '#2ecc71', '#f39c12']

for pore_size, color in zip(pore_sizes_select, colors_pore):
    permeabilities = [carman_kozeny_permeability(pore_size, p) for p in porosity_fine]
    permeabilities_m2 = np.array(permeabilities) * 1e12
    
    ax2.semilogy(porosity_fine * 100, permeabilities_m2, linewidth=3, color=color,
                label=f'd = {pore_size} µm')

ax2.axvspan(50, 70, alpha=0.2, color='green', label='Typical bone scaffold')
ax2.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Permeability (×10⁻¹² m²)', fontsize=12, fontweight='bold')
ax2.set_title('Permeability vs Porosity\nStronger dependence on ε than d',
             fontsize=13, fontweight='bold')
ax2.legend(loc='best', frameon=True, fancybox=True)
ax2.grid(True, alpha=0.3, which='both')

# Plot 3: Oxygen diffusion distance over time
time_hours = np.linspace(0, 48, 100)
diff_distance = oxygen_diffusion_distance(time_hours)

ax3.plot(time_hours, diff_distance, linewidth=3, color='#e74c3c')

# Add critical limits
ax3.axhline(100, color='orange', linestyle='--', linewidth=2,
           label='Cell survival limit (~100 µm)')
ax3.axhline(200, color='red', linestyle='--', linewidth=2,
           label='Necrosis threshold (~200 µm)')

# Mark practical time points
for t in [1, 6, 24]:
    L = oxygen_diffusion_distance(t)
    ax3.plot(t, L, 'o', markersize=12, color='blue',
            markeredgecolor='black', markeredgewidth=2)
    ax3.annotate(f'{t}h: {L:.0f} µm', xy=(t, L), xytext=(10, 10),
                textcoords='offset points', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax3.set_xlabel('Time (hours)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Diffusion Distance (µm)', fontsize=12, fontweight='bold')
ax3.set_title('Oxygen Diffusion Distance\nL = √(4Dt)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)
ax3.set_xlim(0, 48)
ax3.set_ylim(0, 500)

# Plot 4: Flow rate through scaffold (Darcy's law)
# Assume 5 mm diameter, 2 mm thick scaffold with 100 Pa pressure drop
diameter_mm = 5
thickness_mm = 2
pressure_drop_Pa = 100
viscosity_water = 0.001  # Pa·s

area_m2 = np.pi * (diameter_mm/2 * 1e-3)**2
length_m = thickness_mm * 1e-3

pore_sizes_flow = [100, 200, 300, 400]  # µm
porosity_range_flow = np.linspace(0.3, 0.9, 50)

for pore_size, color in zip(pore_sizes_flow, colors_pore):
    flow_rates = []
    for porosity in porosity_range_flow:
        k = carman_kozeny_permeability(pore_size, porosity)
        Q = calculate_flow_rate(k, pressure_drop_Pa, length_m,
                               viscosity_water, area_m2)
        Q_uL_min = Q * 1e9 * 60  # m³/s to µL/min
        flow_rates.append(Q_uL_min)
    
    ax4.plot(porosity_range_flow * 100, flow_rates, linewidth=3, color=color,
            label=f'd = {pore_size} µm')

# Add practical flow rate range
ax4.axhspan(1, 10, alpha=0.2, color='green', label='Practical perfusion range')

ax4.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Flow Rate (µL/min)', fontsize=12, fontweight='bold')
ax4.set_title(f'Perfusion Flow Rate (Darcy\'s Law)\nΔP={pressure_drop_Pa} Pa, L={thickness_mm} mm',
             fontsize=13, fontweight='bold')
ax4.legend(loc='best', frameon=True, fancybox=True)
ax4.grid(True, alpha=0.3)
ax4.set_yscale('log')

plt.tight_layout()
plt.show()

print("\n💡 Mass Transport Insights:")
print("   • Permeability increases with d² (pore size squared)")
print("   • Permeability increases with ε³ (porosity cubed)")
print("   • Porosity has stronger effect than pore size!")
print("   • Oxygen diffuses ~100 µm in 1 hour (without flow)")
print("   • Cells >200 µm from vessels will die without perfusion")
print("   • Higher porosity and larger pores → better perfusion")
print("   • But conflicts with mechanical strength requirements")

## Part 4: Interactive Scaffold Designer

### 🎯 STUDENT TASK 1: Design a Scaffold for Your Application

**Balance mechanical, biological, and transport requirements:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Application
tissue_type = 'bone'  # Options: 'bone', 'cartilage', 'soft_tissue', 'vascular'

# Scaffold design
target_porosity = 0.65        # (try: 0.5, 0.65, 0.8)
pore_diameter = 300           # µm (try: 100, 200, 300, 400)

# Material properties (PCL example)
material_name = 'PCL'
E_material = 400              # MPa
sigma_material = 16           # MPa
rho_material = 1145           # kg/m³

# Scaffold dimensions
scaffold_diameter = 8         # mm
scaffold_thickness = 3        # mm

# ==========================================

# Calculate scaffold properties
E_scaffold_design = gibson_ashby_modulus(E_material, target_porosity)
sigma_scaffold_design = gibson_ashby_strength(sigma_material, target_porosity)
permeability = carman_kozeny_permeability(pore_diameter, target_porosity)
rho_scaffold = rho_material * (1 - target_porosity)

# Calculate flow characteristics
pressure_drop = 100  # Pa (typical for perfusion)
viscosity = 0.001  # Pa·s (water)
area = np.pi * (scaffold_diameter/2 * 1e-3)**2  # m²
length = scaffold_thickness * 1e-3  # m

flow_rate = calculate_flow_rate(permeability, pressure_drop, length, viscosity, area)
flow_rate_uL_min = flow_rate * 1e9 * 60

# Calculate maximum pore spacing for cell survival
max_spacing_um = 2 * oxygen_diffusion_distance(24)  # 24 hours

# Assess pore size
pore_assessment = assess_pore_size(pore_diameter, tissue_type)

# Target tissue properties
tissue_targets = {
    'bone': {'E_min': 50, 'E_max': 500, 'sigma_min': 2, 'sigma_max': 10},
    'cartilage': {'E_min': 5, 'E_max': 20, 'sigma_min': 3, 'sigma_max': 10},
    'soft_tissue': {'E_min': 0.1, 'E_max': 2, 'sigma_min': 0.1, 'sigma_max': 1},
    'vascular': {'E_min': 0.5, 'E_max': 5, 'sigma_min': 0.5, 'sigma_max': 2}
}

targets = tissue_targets[tissue_type]

# Create visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Mechanical Properties', 'Pore Architecture',
                   'Transport Assessment', 'Design Summary'),
    specs=[[{'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'indicator'}, {'type': 'table'}]]
)

# Plot 1: Mechanical properties comparison
fig.add_trace(
    go.Bar(x=['Modulus (MPa)', 'Strength (MPa)'],
          y=[E_scaffold_design, sigma_scaffold_design],
          marker_color=['#3498db', '#e74c3c'],
          text=[f'{E_scaffold_design:.1f}', f'{sigma_scaffold_design:.1f}'],
          textposition='outside',
          name='Scaffold'),
    row=1, col=1
)

# Add target ranges
fig.add_shape(type="rect",
    x0=-0.4, x1=0.4, y0=targets['E_min'], y1=targets['E_max'],
    fillcolor="green", opacity=0.2, line_width=0,
    row=1, col=1
)
fig.add_shape(type="rect",
    x0=0.6, x1=1.4, y0=targets['sigma_min'], y1=targets['sigma_max'],
    fillcolor="green", opacity=0.2, line_width=0,
    row=1, col=1
)

fig.update_yaxes(title_text="Value", type="log", row=1, col=1)

# Plot 2: Pore architecture schematic (simplified 2D cross-section)
# Create a grid of pores
n_pores = 5
spacing = scaffold_diameter * 1000 / n_pores  # µm

for i in range(n_pores):
    for j in range(n_pores):
        # Draw circle for pore
        theta = np.linspace(0, 2*np.pi, 50)
        x_circle = i * spacing + pore_diameter/2 * np.cos(theta)
        y_circle = j * spacing + pore_diameter/2 * np.sin(theta)
        
        fig.add_trace(
            go.Scatter(x=x_circle, y=y_circle, fill='toself',
                      fillcolor='lightblue', line=dict(color='blue', width=1),
                      showlegend=False, hoverinfo='skip'),
            row=1, col=2
        )

fig.update_xaxes(title_text="X position (µm)", row=1, col=2)
fig.update_yaxes(title_text="Y position (µm)", row=1, col=2)

# Plot 3: Transport quality gauge
# Score based on permeability and pore size
if permeability > 1e-10 and pore_diameter > 200:
    transport_score = 90
    transport_status = "Excellent"
    gauge_color = "green"
elif permeability > 1e-11 and pore_diameter > 150:
    transport_score = 70
    transport_status = "Good"
    gauge_color = "yellow"
else:
    transport_score = 40
    transport_status = "Limited"
    gauge_color = "red"

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=transport_score,
        title={'text': f"Transport Quality<br>{transport_status}"},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 50], 'color': "lightgray"},
                {'range': [50, 75], 'color': "lightyellow"},
                {'range': [75, 100], 'color': "lightgreen"}]
        }),
    row=2, col=1
)

# Plot 4: Design summary table
fig.add_trace(
    go.Table(
        header=dict(values=['Parameter', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=11)),
        cells=dict(values=[
            ['Tissue Type', 'Porosity', 'Pore Diameter', 'Modulus',
             'Strength', 'Permeability', 'Flow Rate', 'Density'],
            [tissue_type.title(), f'{target_porosity:.0%}',
             f'{pore_diameter} µm', f'{E_scaffold_design:.1f} MPa',
             f'{sigma_scaffold_design:.1f} MPa',
             f'{permeability*1e12:.2f}×10⁻¹² m²',
             f'{flow_rate_uL_min:.2f} µL/min',
             f'{rho_scaffold:.0f} kg/m³']
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=10))),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False,
                 title_text=f"Scaffold Design Analysis - {tissue_type.title()} Application")
fig.show()

# Detailed text output
print("="*70)
print("SCAFFOLD DESIGN ANALYSIS")
print("="*70)
print(f"\nApplication: {tissue_type.title()}")
print(f"Material: {material_name}")

print(f"\nDesign Parameters:")
print(f"  • Porosity: {target_porosity:.0%}")
print(f"  • Pore diameter: {pore_diameter} µm")
print(f"  • Scaffold size: {scaffold_diameter}×{scaffold_thickness} mm")

print(f"\nMechanical Properties:")
print(f"  • Elastic modulus: {E_scaffold_design:.1f} MPa")
print(f"  • Compressive strength: {sigma_scaffold_design:.1f} MPa")
print(f"  • Density: {rho_scaffold:.0f} kg/m³")
print(f"  • Target range: {targets['E_min']}-{targets['E_max']} MPa (modulus)")

print(f"\nTransport Properties:")
print(f"  • Permeability: {permeability*1e12:.2f}×10⁻¹² m²")
print(f"  • Flow rate (ΔP=100 Pa): {flow_rate_uL_min:.2f} µL/min")
print(f"  • Max O₂ diffusion (24h): {max_spacing_um:.0f} µm")

print(f"\nPore Size Assessment:")
if pore_assessment['suitable']:
    print(f"  ✅ Pore size suitable for {tissue_type}")
else:
    print(f"  ⚠️  Pore size issues:")
    for issue in pore_assessment['issues']:
        print(f"     • {issue}")

print(f"\nMechanical Assessment:")
if targets['E_min'] <= E_scaffold_design <= targets['E_max']:
    print(f"  ✅ Modulus within target range")
elif E_scaffold_design < targets['E_min']:
    print(f"  ⚠️  Modulus too low - consider:")
    print(f"     • Reducing porosity")
    print(f"     • Using stiffer material")
else:
    print(f"  ⚠️  Modulus too high - consider:")
    print(f"     • Increasing porosity")
    print(f"     • Using more compliant material")

if targets['sigma_min'] <= sigma_scaffold_design <= targets['sigma_max']:
    print(f"  ✅ Strength within target range")
elif sigma_scaffold_design < targets['sigma_min']:
    print(f"  ⚠️  Strength too low")
else:
    print(f"  ✅ Strength exceeds minimum (no upper limit concern)")

print(f"\nTransport Assessment:")
if flow_rate_uL_min > 1:
    print(f"  ✅ Adequate perfusion possible")
else:
    print(f"  ⚠️  Low permeability - perfusion may be challenging")
    print(f"     • Consider increasing porosity or pore size")

if pore_diameter > 200:
    print(f"  ✅ Pore size sufficient for vascularization")
elif pore_diameter > 100:
    print(f"  ✓ Pore size adequate for cell migration")
else:
    print(f"  ⚠️  Pore size may restrict cell infiltration")

print(f"\nRecommendations:")
if pore_assessment['recommendations']:
    for rec in pore_assessment['recommendations']:
        print(f"  💡 {rec}")
else:
    print(f"  ✓ Design parameters are well-balanced")

print("\n" + "="*70)

## Part 5: Multi-Objective Design Space Optimization

In [ ]:
# Create 2D design space
porosities_opt = np.linspace(0.4, 0.9, 50)
pore_sizes_opt = np.linspace(100, 500, 50)

P_grid, D_grid = np.meshgrid(porosities_opt, pore_sizes_opt)

# Material: PCL
E_solid_opt = 400  # MPa
sigma_solid_opt = 16  # MPa

# Calculate metrics for each combination
E_grid = np.zeros_like(P_grid)
sigma_grid = np.zeros_like(P_grid)
k_grid = np.zeros_like(P_grid)

for i in range(P_grid.shape[0]):
    for j in range(P_grid.shape[1]):
        porosity = P_grid[i, j]
        pore_size = D_grid[i, j]
        
        E_grid[i, j] = gibson_ashby_modulus(E_solid_opt, porosity)
        sigma_grid[i, j] = gibson_ashby_strength(sigma_solid_opt, porosity)
        k_grid[i, j] = carman_kozeny_permeability(pore_size, porosity)

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Modulus map
contour1 = ax1.contourf(P_grid*100, D_grid, E_grid, levels=20, cmap='viridis')
contour1_lines = ax1.contour(P_grid*100, D_grid, E_grid,
                             levels=[50, 100, 200], colors='white', linewidths=2)
ax1.clabel(contour1_lines, inline=True, fontsize=10, fmt='%d MPa')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Elastic Modulus (MPa)', fontsize=11, fontweight='bold')

# Mark target zones for different tissues
ax1.axhspan(200, 500, alpha=0.15, color='red', label='Bone target')
ax1.axhspan(100, 200, alpha=0.15, color='blue', label='Cartilage target')

ax1.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Pore Diameter (µm)', fontsize=12, fontweight='bold')
ax1.set_title('Elastic Modulus Design Space',
             fontsize=13, fontweight='bold')
ax1.legend(loc='upper right', frameon=True, fancybox=True, fontsize=9)

# Plot 2: Permeability map
k_grid_scaled = k_grid * 1e12  # Scale to 10^-12 m²
contour2 = ax2.contourf(P_grid*100, D_grid, k_grid_scaled, levels=20, cmap='RdYlGn')
contour2_lines = ax2.contour(P_grid*100, D_grid, k_grid_scaled,
                             levels=[1, 10, 100], colors='black', linewidths=2)
ax2.clabel(contour2_lines, inline=True, fontsize=10, fmt='%d')
cbar2 = plt.colorbar(contour2, ax=ax2)
cbar2.set_label('Permeability (×10⁻¹² m²)', fontsize=11, fontweight='bold')

ax2.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Pore Diameter (µm)', fontsize=12, fontweight='bold')
ax2.set_title('Permeability Design Space',
             fontsize=13, fontweight='bold')

# Plot 3: Bone scaffold optimal zone
# Define constraints for bone: 50-500 MPa modulus, >200 µm pores, >50% porosity
bone_zone = (E_grid >= 50) & (E_grid <= 500) & \
            (D_grid >= 200) & (P_grid >= 0.5)

ax3.contourf(P_grid*100, D_grid, bone_zone.astype(float),
            levels=[0.5, 1.5], colors=['white', 'lightgreen'], alpha=0.7)

# Add contour lines for modulus
ax3.contour(P_grid*100, D_grid, E_grid, levels=[50, 100, 200, 500],
           colors=['blue'], linewidths=2, linestyles='dashed')

# Mark optimal points
optimal_designs = [
    {'name': 'Conservative', 'porosity': 0.55, 'pore': 250, 'color': 'blue'},
    {'name': 'Balanced', 'porosity': 0.65, 'pore': 300, 'color': 'green'},
    {'name': 'Aggressive', 'porosity': 0.75, 'pore': 400, 'color': 'orange'}
]

for design in optimal_designs:
    ax3.plot(design['porosity']*100, design['pore'], 'o', markersize=15,
            color=design['color'], markeredgecolor='black', markeredgewidth=2,
            label=design['name'])
    ax3.annotate(design['name'], xy=(design['porosity']*100, design['pore']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor=design['color'],
                         alpha=0.6, edgecolor='black'))

ax3.set_xlabel('Porosity (%)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Pore Diameter (µm)', fontsize=12, fontweight='bold')
ax3.set_title('Bone Scaffold Optimal Zone\n(Green region satisfies all constraints)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='lower right', frameon=True, fancybox=True)

# Plot 4: Trade-off analysis (Pareto front)
# For each pore size, find optimal porosity balancing modulus and permeability
pore_sizes_pareto = np.linspace(150, 450, 20)
pareto_porosities = []
pareto_moduli = []
pareto_permeabilities = []

for pore_size in pore_sizes_pareto:
    # Find porosity that gives E ≈ 100 MPa (target for trabecular bone)
    target_E = 100
    
    # Solve (1-ε)² = E_target/E_solid
    relative_density = np.sqrt(target_E / E_solid_opt)
    porosity = 1 - relative_density
    
    if 0.4 < porosity < 0.9:  # Valid range
        pareto_porosities.append(porosity)
        pareto_moduli.append(target_E)
        k = carman_kozeny_permeability(pore_size, porosity)
        pareto_permeabilities.append(k * 1e12)
    else:
        pareto_porosities.append(np.nan)
        pareto_moduli.append(np.nan)
        pareto_permeabilities.append(np.nan)

# Create twin axes
ax4_twin = ax4.twinx()

line1 = ax4.plot(pore_sizes_pareto, pareto_porosities, 'o-', linewidth=3,
                markersize=8, color='#3498db', label='Porosity')
line2 = ax4_twin.plot(pore_sizes_pareto, pareto_permeabilities, 's-', linewidth=3,
                     markersize=8, color='#2ecc71', label='Permeability')

ax4.set_xlabel('Pore Diameter (µm)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Porosity (for E=100 MPa)', fontsize=12, fontweight='bold', color='#3498db')
ax4_twin.set_ylabel('Permeability (×10⁻¹² m²)', fontsize=12, fontweight='bold', color='#2ecc71')
ax4.set_title('Pareto Front: Mechanical vs Transport\n(Constant Modulus = 100 MPa)',
             fontsize=13, fontweight='bold')
ax4.tick_params(axis='y', labelcolor='#3498db')
ax4_twin.tick_params(axis='y', labelcolor='#2ecc71')

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax4.legend(lines, labels, loc='upper left', frameon=True, fancybox=True)

ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Design Space Insights:")
print("   • Higher porosity → lower modulus but better transport")
print("   • Larger pores → better vascularization but limited effect on modulus")
print("   • Bone scaffolds: sweet spot at 60-70% porosity, 250-350 µm pores")
print("   • Can't optimize all objectives simultaneously (trade-offs)")
print("   • Pareto front shows best compromises for fixed mechanical target")
print("   • Conservative design: prioritize strength (lower porosity)")
print("   • Aggressive design: prioritize biology (higher porosity)")

## Part 6: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 6: KEY LEARNING POINTS")
print("="*80)
print("""
1. POROSITY FUNDAMENTALLY TRADES OFF MECHANICS VS BIOLOGY
   → Higher porosity:
     ✓ Better cell infiltration
     ✓ Better nutrient transport
     ✓ Better vascularization potential
     ✗ Lower mechanical strength
     ✗ Lower elastic modulus
   → Must find optimal balance for each tissue type

2. GIBSON-ASHBY MODEL PREDICTS MECHANICAL PROPERTIES
   → E_scaffold = E_solid · (1-ε)²
   → σ_scaffold = 0.3·σ_solid · (1-ε)^1.5
   → Quadratic dependence on porosity!
   → At 70% porosity: only 9% of solid modulus remains
   → Accurate for open-cell foams and many porous scaffolds

3. CARMAN-KOZENY PREDICTS PERMEABILITY
   → k ∝ d²·ε³/(1-ε)²
   → Cubic dependence on porosity (stronger than mechanics)
   → Quadratic dependence on pore size
   → High permeability essential for perfusion and waste removal

4. PORE SIZE REQUIREMENTS ARE TISSUE-SPECIFIC
   → Cell migration: >50-100 µm
   → Vascularization: >200 µm (critical!)
   → Bone: 200-500 µm optimal
   → Cartilage: 100-200 µm
   → Soft tissue: 50-150 µm
   → Too small → restricted infiltration
   → Too large → weak structure

5. OXYGEN DIFFUSION LIMITS CELL SURVIVAL
   → Diffusion distance: L = √(4Dt)
   → Cells die >200 µm from blood vessels
   → In 24 hours: oxygen diffuses ~350 µm
   → Without vascularization: only outer shell viable
   → Perfusion or rapid vascularization essential!

6. INTERCONNECTIVITY IS CRITICAL
   → Closed pores → no cell migration
   → Must have continuous network of open pores
   → Minimum interconnect size: ~50-100 µm
   → Affects permeability dramatically

7. TISSUE-SPECIFIC DESIGN GUIDELINES
   → Bone:
     • 50-70% porosity
     • 200-500 µm pores
     • E = 50-500 MPa
   → Cartilage:
     • 70-90% porosity
     • 100-200 µm pores
     • E = 5-20 MPa
   → Soft tissue:
     • 80-95% porosity
     • 50-150 µm pores
     • E = 0.1-2 MPa

8. HIERARCHICAL STRUCTURES CAN OPTIMIZE MULTIPLE SCALES
   → Macro-pores (200-500 µm): vascularization, cell migration
   → Micro-pores (1-50 µm): surface area, protein adsorption
   → Nano-features (<1 µm): cell adhesion, signaling
   → Can achieve both strength and biology

9. MECHANICAL MATCHING PREVENTS STRESS SHIELDING
   → Scaffold too stiff → stress shielding → bone resorption
   → Scaffold too soft → insufficient support → implant failure
   → Target: match native tissue modulus
   → Degradation should track tissue formation

10. DESIGN STRATEGY
    1. Identify target tissue and mechanical requirements
    2. Choose material with appropriate E_solid
    3. Calculate required porosity from Gibson-Ashby
    4. Select pore size for tissue type (>200 µm for bone)
    5. Verify permeability is adequate (>10^-11 m²)
    6. Check oxygen diffusion constraints
    7. Plan for vascularization or perfusion
    8. Consider hierarchical structure if needed
    9. Prototype and test!
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**Derive the Gibson-Ashby equation E/E₀ = (1-ε)² for open-cell foams. Start from considering bending of struts under load. Why is the exponent 2 instead of 1?**

### Question 2  
**You're designing a bone scaffold with PCL (E = 400 MPa). Target: trabecular bone properties (E = 100 MPa). Calculate the required porosity. Then calculate the pore diameter needed to achieve permeability k = 10⁻¹¹ m². Is this pore size appropriate for bone?**

### Question 3
**Explain why the Carman-Kozeny equation shows cubic dependence on porosity (ε³) while Gibson-Ashby shows quadratic dependence (ε²). What does this mean for design trade-offs?**

### Question 4
**A 5 mm thick cartilage scaffold has 200 µm pores and 75% porosity. Without vascularization, how long before cells in the center begin to die? Show your calculation using the diffusion equation. What strategies could prevent this?**

### Question 5
**Compare two bone scaffold designs: Design A (60% porosity, 250 µm pores) vs Design B (75% porosity, 350 µm pores). Calculate and compare: (1) elastic modulus, (2) permeability, (3) suitability for vascularization. Which would you choose and why?**

## 📚 Additional Challenges (Optional)

### Challenge 1: Anisotropic Scaffolds
How would you modify the Gibson-Ashby equations for aligned pore structures? What advantages might anisotropy provide for bone or muscle tissue engineering?

### Challenge 2: Degradation Dynamics
Model how scaffold properties change as a biodegradable polymer degrades. How should you design initial porosity to maintain mechanical support during tissue formation?

### Challenge 3: Multiscale Optimization
Design a hierarchical scaffold with three pore sizes: macro (300 µm), micro (30 µm), and nano (3 µm). Calculate effective porosity and mechanical properties. What fabrication methods could create this?

### Challenge 4: Perfusion Bioreactor Design
Design a perfusion system for a 10 mm diameter scaffold. Calculate required flow rate to deliver sufficient oxygen to all cells. What pressure drop is needed?

## 🎯 Congratulations!

You've completed Exercise 6: Scaffold Architecture and Porosity Design!

**You now understand:**
- ✓ Gibson-Ashby model for mechanical property prediction
- ✓ Carman-Kozeny equation for permeability
- ✓ Oxygen diffusion constraints on scaffold design
- ✓ Tissue-specific pore size requirements
- ✓ Multi-objective optimization and trade-offs
- ✓ Practical design strategies for different tissues

**Next Steps:**
- Continue to Exercise 7: Bioink Rheology and Printability
- Review Chapter 4.3 for scaffold architecture principles
- Explore research papers on hierarchical porous structures

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*